In [1]:
%load_ext autoreload
%autoreload 2

*Make sure to have the right kernel selected!*

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

%metal_heading Welcome to Qiskit Metal!

Welcome to Qiskit Metal! 

For this example tutorial, we will attempt to create a multi qubit chip with a variety of components. We will want to generate the layout, simulate/analyze and tune the chip to hit the parameters we are wanting, finally rendering to a GDS file.

One could generate subsections of the layout and tune individual components first, but in this case we will create all of the layout. We will be using both transmon pockets and crossmons, meandered and simple transmission lines, capacitive couplers, and launchers for wirebond connections. So we will import these, and also create a design instance and launch the GUI.

### Layout

In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL

from qiskit_metal.qlibrary.qubits.tunable_coupler_01 import TunableCoupler01

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
from qiskit_metal.qlibrary.interconnects.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.interconnects.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.connectors.cpw_finger_cap import CPWFingerCap
from qiskit_metal.qlibrary.connectors.cpw_t_finger_cap import CPWTFingerCap
from qiskit_metal.qlibrary.connectors.cpw_hanger_t import CPWHangerT

from qiskit_metal.qlibrary.passives.launchpad_wb import LaunchpadWirebond
from qiskit_metal.qlibrary.passives.launchpad_wb_coupled import LaunchpadWirebondCoupled

In [4]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

Since we are likely to be making many changes while tuning and modifying our design, we will enable overwriting. We can also check all of the chip properties to see if we want to change the size or any other parameter.

In [5]:
design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [67]:
design.chips.main.size.size_x = '11mm'
design.chips.main.size.size_y = '9mm'

#### The Qubits

We will add a collection of qubits. First we will place a transmon pocket with six connection pads. We can see any options the qubit qcomponent has to figure out what we might want to modify when creating the component. This will include the components default options (which the component designer included) as well as renderer options (which are added based on what renderers are present in Metal).

In [6]:
TransmonPocket6.get_template_options(design)

{'pos_x': '0um',
 'pos_y': '0um',
 'connection_pads': {},
 '_default_connection_pads': {'pad_gap': '15um',
  'pad_width': '125um',
  'pad_height': '30um',
  'pad_cpw_shift': '0um',
  'pad_cpw_extent': '25um',
  'cpw_width': '10um',
  'cpw_gap': '6um',
  'cpw_extend': '100um',
  'pocket_extent': '5um',
  'pocket_rise': '0um',
  'loc_W': '+1'},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'orientation': '0',
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction'}

In [7]:
options =  dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    connection_pads=dict(
        readout = dict(loc_W=0, loc_H=-1, pad_width = '80um', pad_gap = '50um'),
        bus_01 = dict(loc_W=-1, loc_H=-1, pad_width = '60um', pad_gap = '10um'),
        bus_02 = dict(loc_W=-1, loc_H=+1, pad_width = '60um', pad_gap = '10um'),
        bus_03 = dict(loc_W=0, loc_H=+1, pad_width = '90um', pad_gap = '30um'),
        bus_04 = dict(loc_W=+1, loc_H=+1, pad_width = '60um', pad_gap = '10um'),
        bus_05 = dict(loc_W=+1, loc_H=-1, pad_width = '60um', pad_gap = '10um')        
    ))

Q_main = TransmonPocket6(design,'Q_Main', options = dict(
        pos_x='0mm', 
        pos_y='-1mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options))

gui.rebuild()
gui.autoscale()

We then will add a mixture of additional qubits. This is not (though do not let me stop any experimental investigation) a design one would normally create for any experiment of computational purpose, but allows for having a mixture of different components on one chip.

In [8]:
TransmonCrossFL.get_template_options(design)

{'pos_x': '0um',
 'pos_y': '0um',
 'connection_pads': {},
 '_default_connection_pads': {'connector_type': '0',
  'claw_length': '30um',
  'ground_spacing': '5um',
  'claw_width': '10um',
  'claw_gap': '6um',
  'connector_location': '0'},
 'cross_width': '20um',
 'cross_length': '200um',
 'cross_gap': '20um',
 'orientation': '0',
 'layer': '1',
 'make_fl': True,
 'fl_options': {'t_top': '15um',
  't_offset': '0um',
  't_inductive_gap': '3um',
  't_width': '5um',
  't_gap': '3um'},
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction'}

We will add two crossmons with flux lines to the west side of the chip, which we will couple to each other using a tunable coupler. To make sure the various readout and control lines will have space to connect to launchers at the chip edge, we have to be mindful of where we place them, and making sure we have enough space for routing while avoiding cross talk.

In [9]:
Q1 = TransmonCrossFL(design, 'Q1', options = dict(pos_x = '-2.75mm', pos_y='-1.8mm',
                                                 connection_pads = dict(
                                                     bus_01 = dict(connector_location = '180',claw_length ='95um'),
                                                     readout = dict(connector_location = '0')),
                                                 fl_options = dict()))

Q2 = TransmonCrossFL(design, 'Q2', options = dict(pos_x = '-2.75mm', pos_y='-1.2mm', orientation = '180',
                                                 connection_pads = dict(
                                                     bus_02 = dict(connector_location = '0',claw_length ='95um'),
                                                     readout = dict(connector_location = '180')),
                                                 fl_options = dict()))


tune_c_Q12 = TunableCoupler01(design,'Tune_C_Q12', options = dict(pos_x = '-2.81mm', pos_y = '-1.5mm', 
                                                                  orientation=90, c_width='500um'))

gui.rebuild()
gui.autoscale()

We then will add three transmon pockets to the north side of the chip, with the intention of having them in a linear series of coupling to each other, as well as the 'main' qubit to the south.

In [10]:
Q3 = TransmonPocket6(design,'Q3', options = dict(
        pos_x='-3mm', 
        pos_y='0.5mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        connection_pads = dict(
            bus_03 = dict(loc_W=0, loc_H=-1, pad_width = '80um', pad_gap = '15um'),
            bus_q3_q4 = dict(loc_W=1, loc_H=-1, pad_width = '80um', pad_gap = '15um'),
            readout = dict(loc_W=0, loc_H=1, pad_width = '80um', pad_gap = '50um'))))

Q4 = TransmonPocket6(design,'Q4', options = dict(
        pos_x='0mm', 
        pos_y='1mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        connection_pads = dict(
            bus_04 = dict(loc_W=0, loc_H=-1, pad_width = '80um', pad_gap = '15um'),
            bus_q3_q4 = dict(loc_W=-1, loc_H=-1, pad_width = '80um', pad_gap = '15um'),
            bus_q4_q5 = dict(loc_W=1, loc_H=-1, pad_width = '80um', pad_gap = '15um'),
            readout = dict(loc_W=0, loc_H=1, pad_width = '80um', pad_gap = '50um'))))

Q5 = TransmonPocket6(design,'Q5', options = dict(
        pos_x='3mm', 
        pos_y='0.5mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        connection_pads = dict(
            bus_05 = dict(loc_W=0, loc_H=-1, pad_width = '80um', pad_gap = '15um'),
            bus_q4_q5 = dict(loc_W=-1, loc_H=-1, pad_width = '80um', pad_gap = '15um'),
            readout = dict(loc_W=0, loc_H=1, pad_width = '80um', pad_gap = '50um'))))

#### The Busses

We now couple the qubits to each other, primarily using RouteMeander. Although one needs to run simulations to properly tune the line lengths for target frequencies, an initial estimate could be determined from the below method;

In [11]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

As we are not worried about a creating a functional chip in this tutorial, we will give the resonators somewhat arbitraty lengths. First coupling the two crossmons to Q_Main.

In [12]:
bus_01 = RouteMeander(design,'Bus_01', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Main',
                                                    pin='bus_01'),
                                                end_pin=Dict(
                                                    component='Q1',
                                                    pin='bus_01')
                                            ),
                                            lead=Dict(
                                                start_straight='125um',
                                                end_straight = '225um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '1305um'),
                                            fillet = "99um",
                                            total_length = '6mm'))

bus_02 = RouteMeander(design,'Bus_02', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Main',
                                                    pin='bus_02'),
                                                end_pin=Dict(
                                                    component='Q2',
                                                    pin='bus_02')
                                            ),
                                            lead=Dict(
                                                start_straight='325um',
                                                end_straight = '125um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '450um'),
                                            fillet = "99um",
                                            total_length = '6.4mm'))


gui.rebuild()

Then the three transmon pockets on the north side to Q_Main.

In [13]:
bus_03 = RouteMeander(design,'Bus_03', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Main',
                                                    pin='bus_03'),
                                                end_pin=Dict(
                                                    component='Q3',
                                                    pin='bus_03')
                                            ),
                                            lead=Dict(
                                                start_straight='225um',
                                                end_straight = '25um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '50um'),
                                            fillet = "99um",
                                            total_length = '6.8mm'))

#To help set the right spacing, jogs can be used to set some initially controlled routing paths
from collections import OrderedDict
jogs_start = OrderedDict()
jogs_start[0] = ["L", '250um']
jogs_start[1] = ["R", '200um']
#jogs_start[2] = ["L", '200um']

jogs_end = OrderedDict()
jogs_end[0] = ["L", '600um']
#jogs_end[1] = ["L", '800um']

bus_04 = RouteMeander(design,'Bus_04', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Main',
                                                    pin='bus_04'),
                                                end_pin=Dict(
                                                    component='Q4',
                                                    pin='bus_04')
                                            ),
                                            lead=Dict(
                                                start_straight='225um',
                                                #end_straight = '25um',
                                                start_jogged_extension=jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '150um'),
                                            fillet = "99um",
                                            total_length = '7.2mm'))

bus_05 = RouteMeander(design,'Bus_05', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Main',
                                                    pin='bus_05'),
                                                end_pin=Dict(
                                                    component='Q5',
                                                    pin='bus_05')
                                            ),
                                            lead=Dict(
                                                start_straight='225um',
                                                end_straight = '25um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '50um'),
                                            fillet = "99um",
                                            total_length = '7.6mm'))

gui.rebuild()

Finally the three transmon pockets on the north side to each other. This concludes the interconnectivity between the qubits.

In [14]:
bus_q3_q4 = RouteMeander(design,'Bus_Q3_Q4', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q3',
                                                    pin='bus_q3_q4'),
                                                end_pin=Dict(
                                                    component='Q4',
                                                    pin='bus_q3_q4')
                                            ),
                                            lead=Dict(
                                                start_straight='125um',
                                                end_straight = '125um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '50um'),
                                            fillet = "99um",
                                            total_length = '6.4mm'))

bus_q4_q5 = RouteMeander(design,'Bus_Q4_Q5', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q4',
                                                    pin='bus_q4_q5'),
                                                end_pin=Dict(
                                                    component='Q5',
                                                    pin='bus_q4_q5')
                                            ),
                                            lead=Dict(
                                                start_straight='125um',
                                                end_straight = '25um'
                                            ),
                                            meander=Dict(
                                                asymmetry = '50um'),
                                            fillet = "99um",
                                            total_length = '6.8mm'))

gui.rebuild()

#### The Readouts and Control Lines

The intention for this design is to have the three north transmon pockets be multiplexed to one readout line. The crossmons to their own readouts, as well as Q_Main. The tunable coupler, and the two crossmons also have flux lines which need to be connected to launchers.
First we will place the wirebond launchers at the edges of the chip.

In [49]:
launch_qmain_read = LaunchpadWirebond(design, 'Launch_QMain_Read', options = dict(pos_x = '2mm', pos_y ='-4mm', orientation = '90'))

launch_q1_fl = LaunchpadWirebond(design, 'Launch_Q1_FL', options = dict(pos_x = '0mm', pos_y ='-4mm', orientation = '90', 
                                                                        trace_width = '5um',
                                                                        trace_gap = '3um',))
launch_q1_read = LaunchpadWirebondCoupled(design, 'Launch_Q1_Read', options = dict(pos_x = '-2mm', pos_y ='-4mm', orientation = '90'))

launch_tcoup_fl = LaunchpadWirebond(design, 'Launch_TuneC_FL', options = dict(pos_x = '-4mm', pos_y ='-4mm', orientation = '90',
                                                                             trace_width = '5um',
                                                                             trace_gap = '3um',))
launch_tcoup_read = LaunchpadWirebondCoupled(design, 'Launch_TuneC_Read', options = dict(pos_x = '-5mm', pos_y ='-3mm', orientation = '0'))

launch_q2_read = LaunchpadWirebondCoupled(design, 'Launch_Q2_Read', options = dict(pos_x = '-5mm', pos_y ='-1mm', orientation = '0'))
launch_q2_fl = LaunchpadWirebond(design, 'Launch_Q2_FL', options = dict(pos_x = '-5mm', pos_y ='1mm', orientation = '0',
                                                                       trace_width = '5um',
                                                                        trace_gap = '3um',))

launch_nw = LaunchpadWirebond(design, 'Launch_NW',options = dict(pos_x = '-5mm', pos_y='3mm', orientation=0))
launch_ne = LaunchpadWirebond(design, 'Launch_NE',options = dict(pos_x = '5mm', pos_y='3mm', orientation=180))


gui.rebuild()

We then will add in the readout resonators for Q_Main, Q1, Q2 and the tuneable coupler.
We will add a finger capacitor for the Q_Main readout, instead of just using the LaunchpadWirebondCoupled.

In [42]:
#Main Readout


read_q_main_cap = CPWFingerCap(design,'Read_Q_Main_Cap', options = dict(pos_x = '2mm', pos_y ='-3.5mm', orientation = '0'))

jogs_end = OrderedDict()
jogs_end[0] = ["L", '600um']

jogs_start = OrderedDict()
jogs_start[0] = ["L", '250um']

read_q_main = RouteMeander(design,'Read_Q_Main', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q_Main',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Read_Q_Main_Cap',
                                                    pin='north_end')
                                            ),
                                            lead=Dict(
                                                start_straight='725um',
                                                end_straight = '625um',
                                                start_jogged_extension = jogs_start,
                                                end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '50um'),
                                            fillet = "99um",
                                            total_length = '5.6mm'))

read_q_main_cap_launch = RoutePathfinder(design, 'Read_Q_Main_Cap_Launch', options = dict(
                                                    pin_inputs = dict(
                                                        start_pin=Dict(
                                                    component='Read_Q_Main_Cap',
                                                    pin='south_end'),
                                                end_pin=Dict(
                                                    component='Launch_QMain_Read',
                                                    pin='tie')),
                                                lead=Dict(
                                                start_straight='0um',
                                                end_straight = '0um',
                                                #start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            )))
    


gui.rebuild()

In [17]:
#Crossmon's Readouts

jogs_end = OrderedDict()
jogs_end[0] = ["L", '600um']

jogs_start = OrderedDict()
jogs_start[0] = ["L", '250um']

read_q1 = RouteMeander(design,'Read_Q1', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Launch_Q1_Read',
                                                    pin='tie')
                                            ),
                                            lead=Dict(
                                                start_straight='250um',
                                                end_straight = '25um',
                                                #start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '50um'),
                                            fillet = "99um",
                                            total_length = '6.8mm'))


jogs_end = OrderedDict()
jogs_end[0] = ["L", '600um']

jogs_start = OrderedDict()
jogs_start[0] = ["L", '250um']

read_tunec = RouteMeander(design,'Read_TuneC', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Tune_C_Q12',
                                                    pin='Control'),
                                                end_pin=Dict(
                                                    component='Launch_TuneC_Read',
                                                    pin='tie')
                                            ),
                                            lead=Dict(
                                                start_straight='1525um',
                                                end_straight = '125um',
                                                #start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '50um'),
                                            fillet = "99um",
                                            total_length = '5.8mm'))

jogs_end = OrderedDict()
jogs_end[0] = ["L", '600um']

jogs_start = OrderedDict()
jogs_start[0] = ["L", '250um']



read_q2 = RouteMeander(design,'Read_Q2', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Launch_Q2_Read',
                                                    pin='tie')
                                            ),
                                            lead=Dict(
                                                start_straight='350um',
                                                end_straight = '0um',
                                                #start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '-450um'),
                                            fillet = "99um",
                                            total_length = '5.4mm'))


gui.rebuild()

Finishing off this section of the chip by connecting the flux lines to appropraite wirebond launch pads.

In [31]:
#Crossmon flux lines

flux_line_Q1 = RoutePathfinder(design,'Flux_Line_Q1', options = dict(
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Q1_FL',
                                                    pin='tie')),
                                            fillet = '99um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors
                                            ))


jogs_start = OrderedDict()
jogs_start[0] = ["L", '750um']

flux_line_tunec = RoutePathfinder(design,'Flux_Line_TuneC', options = dict(
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Tune_C_Q12',
                                                    pin='Flux'),
                                                end_pin=Dict(
                                                    component='Launch_TuneC_FL',
                                                    pin='tie')),
                                            lead=Dict(
                                                start_straight='875um',
                                                end_straight = '350um',
                                                start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '99um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors
                                            ))


jogs_start = OrderedDict()
jogs_start[0] = ["L", '525um']
jogs_start[1] = ["R", '625um']

flux_line_Q2 = RoutePathfinder(design,'Flux_Line_Q2', options = dict(
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='flux_line'),
                                                end_pin=Dict(
                                                    component='Launch_Q2_FL',
                                                    pin='tie')),
                                            lead=Dict(
                                                start_straight='175um',
                                                end_straight = '150um',
                                                start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            fillet = '99um',
                                            trace_width = '5um',
                                            trace_gap = '3um',
                                            #anchors = anchors
                                            ))

gui.rebuild()

Shifting our focus now to the three transmon pockets in the north. As we want these to be multiplexed to a single readout line, we will add in a few three port components, such as the CPWHangerT and CPWTFingerCap.
Q3 will have an inductive coupling to the readout line (as we want a lambda/4 resonator), Q4 will have a simple gap capacitor, and Q5 will have an interdigitated capacitor.

In [59]:
q3_read_T = CPWHangerT(design,'Q3_Read_T', options=dict(pos_x = '-3mm', pos_y = '3mm', 
                                                        orientation = '0', 
                                                        coupling_length = '200um',
                                                        open_termination = False))
#We use finger count to set the width of the gap capacitance, -> N*cap_width + (N-1)*cap_gap
q4_read_T = CPWTFingerCap(design,'Q4_Read_T', options=dict(pos_x = '0mm', pos_y = '3mm', 
                                                           orientation = '0', 
                                                           finger_length = '0um', 
                                                           finger_count = '8'))
q5_read_T = CPWTFingerCap(design,'Q5_Read_T', options=dict(pos_x = '3mm', pos_y = '3mm', 
                                                           orientation = '0', 
                                                           finger_length = '50um', 
                                                           finger_count = '11'))

gui.rebuild()

05:17PM 49s WARNING [check_lengths]: For path table, component=Read_Q3, key=trace has short segments that could cause issues with fillet. Values in (11-12)  are index(es) in shapley geometry.
05:17PM 49s WARNING [check_lengths]: For path table, component=Read_Q3, key=cut has short segments that could cause issues with fillet. Values in (11-12)  are index(es) in shapley geometry.


We add in the readout resonators to each respective qubit.

In [62]:
read_q3 = RouteMeander(design,'Read_Q3', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q3',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Q3_Read_T',
                                                    pin='second_end')
                                            ),
                                            lead=Dict(
                                                start_straight='150um',
                                                end_straight = '150um',
                                                #start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '0um'),
                                            fillet = "99um",
                                            total_length = '3mm'))

read_q4 = RouteMeander(design,'Read_Q4', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q4',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Q4_Read_T',
                                                    pin='second_end')
                                            ),
                                            lead=Dict(
                                                start_straight='125um',
                                                end_straight = '125um',
                                                #start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '0um'),
                                            fillet = "99um",
                                            total_length = '5.8mm'))

read_q5 = RouteMeander(design,'Read_Q5', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q5',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Q5_Read_T',
                                                    pin='second_end')
                                            ),
                                            lead=Dict(
                                                start_straight='125um',
                                                end_straight = '125um',
                                                #start_jogged_extension = jogs_start,
                                                #end_jogged_extension = jogs_end
                                            ),
                                            meander=Dict(
                                                asymmetry = '0um'),
                                            fillet = "99um",
                                            total_length = '5.4mm'))

gui.rebuild()

We complete the layout by connecting the multiplexed readout line to the launchpads on either side of the chip.

In [65]:
mp_tl_01 = RoutePathfinder(design, 'ML_TL_01', options = dict(
                                                    pin_inputs = dict(
                                                    start_pin=Dict(
                                                        component='Launch_NW',
                                                        pin='tie'),
                                                    end_pin=Dict(
                                                        component='Q3_Read_T',
                                                        pin='prime_start'))
                                            ))

mp_tl_02 = RoutePathfinder(design, 'ML_TL_02', options = dict(
                                                    pin_inputs = dict(
                                                    start_pin=Dict(
                                                        component='Q3_Read_T',
                                                        pin='prime_end'),
                                                    end_pin=Dict(
                                                        component='Q4_Read_T',
                                                        pin='prime_start'))
                                            ))

mp_tl_03 = RoutePathfinder(design, 'ML_TL_03', options = dict(
                                                    pin_inputs = dict(
                                                    start_pin=Dict(
                                                        component='Q4_Read_T',
                                                        pin='prime_end'),
                                                    end_pin=Dict(
                                                        component='Q5_Read_T',
                                                        pin='prime_start'))
                                            ))

mp_tl_04 = RoutePathfinder(design, 'ML_TL_04', options = dict(
                                                    pin_inputs = dict(
                                                    start_pin=Dict(
                                                        component='Q5_Read_T',
                                                        pin='prime_end'),
                                                    end_pin=Dict(
                                                        component='Launch_NE',
                                                        pin='tie'))
                                            ))

gui.rebuild()

With this, we have completed the construction of our layout. 

Now, anyone familiar with chip design might find some of the location choices to be sub-optimal, with large sections of your chip left unused, or perhaps some CPW transmission lines running a bit closer to each other than would be ideal for avoiding cross talk concerns. These could be address by shifting the origin of your chip, or modifying component options to better compact your layout and alleviate crosstalk concerns.

For this tutorial, we aren't too concerned how much space we may use up on our fictional chip, so we will instead continue on to analysis and tuning.